### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/

In [18]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np
from datetime import datetime

### List of URLs input to search short list manually entered

In [ ]:
urls = ['https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006404',
       'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006406?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006401?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006417?emailalert=true'
       ]

### Find URLS from saved email in html - save as from outlook in htm format

In [26]:
input = (r'IJSEMemail7.htm')

In [27]:
#with open ('IJSEMemail1.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
print(soup.prettify())

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns:ns0="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Word.Document" name="ProgId"/>
  <meta content="Microsoft Word 15" name="Generator"/>
  <meta content="Microsoft Word 15" name="Originator"/>
  <link href="IJSEMemail7_files/filelist.xml" rel="File-List"/>
  <link href="IJSEMemail7_files/editdata.mso" rel="Edit-Time-Data"/>
  <!--[if !mso]>
<style>
v\:* {behavior:url(#default#VML);}
o\:* {behavior:url(#default#VML);}
w\:* {behavior:url(#default#VML);}
.shape {behavior:url(#default#VML);}
</style>
<![endif]-->
  <!--[if gte mso 9]><xml>
 <o:OfficeDocumentSettings>
  <o:AllowPNG/>
 </o:OfficeDocumentSettings>
</xml><![endif]-->
  <link href="IJSEMemail7_files/them

In [28]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
    #print(url)
#print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e]

#print(dois)
#print(urls)

### Selenium web scraper - title and author note (footnote 1), some of these functions can be turned off following additional testing. Currently not using this function

In [5]:
#Function to scrape headlines using Selenium
def scrape_with_selenium(url, counter, doi):
    global title
    global footnote
    global description
    title = []
    description = None
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)

    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
    #for element in driver.find_elements(By.CLASS_NAME, "author-footnote-below-abstract"):get.text() #retrieves all footnotes
    for element in driver.find_elements(By.ID, "FN1"): #retrieves just footnote 1
        #print(element.text)
        footnote = element.text
        #print(footnote)
        #print(type(footnote))
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        print(doi)
    
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        print(description)
        if "Description of" in description: 
            print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
    #Close the browser window
    driver.quit()
    return title, footnote, description, doi



### Beautifulsoup webscraper - returns abstract, can be turned off following additional testing. This function is called but we are not longer using the output.

In [38]:
def scrape_with_beautifulsoup(url):
    global abstract_text
    abstract_text = None
    response = requests.get(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    soup = BeautifulSoup(response.text, "lxml")
    #print(soup.prettify())
    abstract = soup.find(class_= "articleabstract")
    if abstract is not None:
    #print(abstract.get_text())
        abstract_text = abstract.get_text()
        return abstract_text
    else:
        return

### Main body 
### Selenium to extract data from html
### https://nameparser.readthedocs.io/en/latest/index.html to find the last name

In [40]:
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'Authority', 'DOI', 'URL'])
pd.set_option('display.max_columns', None)
for url in urls:
    counter = 1
    strains = []
    accessions = []
    doi = []
    author = []
    date = []
    year = []
    author_raw = []
    author = []
    name1 = []
    name2 = []
    author_count = []
    authority = []
    description = None
    #scrape_with_selenium(url, counter)
    scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        #print(doi)
    
    #find publication year
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[3]/span/span[2]"):
        date = element.text
        year = date[-4:]
        #print(year)
    
    #find authors
    for element in driver.find_elements(By.XPATH, "//*[@id='bellowheadercontainer']/main/div[2]/div/ul/li[1]/span"):
        author_raw = element.text
        print(author_raw)
        author = re.sub(r"[\d+]+",'',author_raw)
        author = re.sub(r"†",'',author)
        author = re.sub(r" and ",',',author)
        author = re.sub(r",,",',',author)  
        author = author.split(',')
        author[:] = [item for item in author if item != '']
        print(author)
        author_count = len(author)
        print(author_count)
        if author_count == 1:
            name1 = (str(author[0])) 
            name1 = HumanName(name1)
            authority = name1.last + ' ' +str(year)
            print(authority)
        elif author_count ==2:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            name2 = (str(author[1]))
            name2 = HumanName(name2)
            authority = name1.last + ' and ' + name2.last + ' ' +str(year)
            print(authority)
        else:
            name1 = (str(author[0]))
            name1 = HumanName(name1)
            authority = name1.last + ' et al. ' +str(year)
            print(authority)
    
    #extract data from the species description
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description: 
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
                #find the organism names    
                match = [r'(\S+\s+){2}(?=sp. nov.)', r'(\S+\s+){2}(?=nom. nov.)', r'(\S+\s+){2}(?=SP. NOV.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                orgname = [m.group() for m in regex.finditer(description)]
                print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    print('accessions', accessions)
    
                #find the strains
                #strainpattern = [r'(?<=type strain).*'] 
                strainpattern = [r'(?<=type strain).*?(?=\))']
                regex = re.compile(r'\b(' + '|'.join(strainpattern) + r')\b')
                if description is not None:
                    strains = [m.group() for m in regex.finditer(description)]
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, authority, doi, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK')
    
#Close the browser window
    driver.quit()    

Australia
['Australia']
1
 []
Neisseria leonii sp. nov., isolated from the nose, lung, and liver of rabbits
Martin Boutroux1,†, Sandrine Favre-Rochex2, Olivier Gorgette3, Gérald Touak2, Estelle Mühle2, Christiane Bouchier2, Olivier Chesneau2, Frédéric J. Veyrier4, Dominique Clermont2 and Praveen Rahi2
['Martin Boutroux', ' Sandrine Favre-Rochex', ' Olivier Gorgette', ' Gérald Touak', ' Estelle Mühle', ' Christiane Bouchier', ' Olivier Chesneau', ' Frédéric J. Veyrier', ' Dominique Clermont', 'Praveen Rahi']
10
Boutroux et al. 2024
orgname ['Neisseria leonii ']
accessions ['CP145606', 'OQ121838']
strain names [' of the species is 3986T (=R726T=CIP 109994T=LMG 32907T']
orgname ['Neisseria leonii ']
accessions []
strain names []
BREAK
Sphingobacterium tenebrionis sp. nov., isolated from intestine of mealworm
Chengsong Zhang1,2, Guoqiang Zhang2, Yuexing Chen2, Shanmin Zheng2, Jieke Du2, Zhiyi Zhao2, Yushuo Zhao2, Ning Wang2, Cheng Chen2, Zhengquan Gao3, Shengying Li2,4 and Kun Liu2
['Cheng

Maribellus mangrovi sp. nov., an iron-reducing bacterium isolated from mangrove sediment
Hongxuan Wang1, Xiaoxuan Song2, Zhenxing Xu1,2,3 and Zong-Jun Du1,2,3
['Hongxuan Wang', ' Xiaoxuan Song', ' Zhenxing Xu', 'Zong-Jun Du']
4
Wang et al. 2024
orgname ['Maribellus mangrovi ']
accessions ['PP485723', 'JBBIXN000000000']
strain names [' from mangrove sediment', ', FJH33T (=KCTC 102210T=MCCC 1H01459T']
orgname ['Maribellus mangrovi ']
accessions []
strain names []
BREAK


In [41]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [','.join(map(str, l)) for l in pub_df['Strains']]
pub_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,[Neisseria leonii ],"[CP145606, OQ121838]",of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
1,[Neisseria leonii ],[],,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,[Sphingobacterium tenebrionis ],"[PP092110, JAYLLN000000000]",", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
3,[Sphingobacterium tenebrionis ],[],,Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,[Phenylobacterium montanum ],"[PP386696, CP073078, CP073079, SAMN18580201]",is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
5,[Phenylobacterium montanum ],[],,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
6,[Lacrimispora brassicae ],"[LC715209, BRYH00000000]",is YZC6T (=MAFF 212518T=JCM 32810T=DSM 112100T,Kobayashi et al. 2024,https://doi.org/10.1099/ijsem.0.006456,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2

In [43]:
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
pub_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,[Neisseria leonii ],"[CP145606, OQ121838]",of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,[Sphingobacterium tenebrionis ],"[PP092110, JAYLLN000000000]",", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,[Phenylobacterium montanum ],"[PP386696, CP073078, CP073079, SAMN18580201]",is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
6,[Lacrimispora brassicae ],"[LC715209, BRYH00000000]",is YZC6T (=MAFF 212518T=JCM 32810T=DSM 112100T,Kobayashi et al. 2024,https://doi.org/10.1099/ijsem.0.006456,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006456%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691389402%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=gg%2F%2BVK5LCWfAobEjmMSvYc9nLNR0o7qJGCizyOIeyks%3D&reserved=0
8,[Pedobacter faecalis ],"[OQ406256, JARXOW000000000]","can assimilate d-glucose, d-mannose, N-acetyl-glucosamine, and maltose, but not l-arabinose, d-mannitol, potassium gluconate, capric acid, adipic acid, malic acid, trisodium citrate, or phenylacetic acid. Furthermore, enzyme activities of alkaline phosphatase, leucine arylamidase, cystine arylamidase, acid phosphatase, naphthol-AS-BI-phosphohydrolase, N-acetyl-β-glucosaminidase, and valine arylamidase is detected in strain ELA7T. In contrast, esterase (C4, is not able to produce indole and lacks the capability to reduce nitrate to nitrite. Menaquinone-7 (MK-7,, ELA7T (=KACC 23137T=JCM 36003T",Park et al. 2024,https://doi.org/10.1099/ijsem.0.006454,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006454%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691400034%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=RVclGZMiIi4JTZzzX2ZsPi8DE53fWVfwULQ0lLEvSK8%3D&reserved=0
10,[Methylobacterium amylolyticum ],"[OR473296, JAUYZI000000000]",is NEAU-140T (=MCCC 1K08801T=DSM 110568T,Zhang et al. 2024,https://doi.org/10.1099/

In [44]:
#try drop duplicate accessions here
pub_df = pub_df.drop_duplicates(subset='Accessions', keep="first")
pub_df.explode(['PublishedName']).reset_index(drop=True)
pub_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,[Neisseria leonii ],"[CP145606, OQ121838]",of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,[Sphingobacterium tenebrionis ],"[PP092110, JAYLLN000000000]",", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,[Phenylobacterium montanum ],"[PP386696, CP073078, CP073079, SAMN18580201]",is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
6,[Lacrimispora brassicae ],"[LC715209, BRYH00000000]",is YZC6T (=MAFF 212518T=JCM 32810T=DSM 112100T,Kobayashi et al. 2024,https://doi.org/10.1099/ijsem.0.006456,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006456%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691389402%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=gg%2F%2BVK5LCWfAobEjmMSvYc9nLNR0o7qJGCizyOIeyks%3D&reserved=0
8,[Pedobacter faecalis ],"[OQ406256, JARXOW000000000]","can assimilate d-glucose, d-mannose, N-acetyl-glucosamine, and maltose, but not l-arabinose, d-mannitol, potassium gluconate, capric acid, adipic acid, malic acid, trisodium citrate, or phenylacetic acid. Furthermore, enzyme activities of alkaline phosphatase, leucine arylamidase, cystine arylamidase, acid phosphatase, naphthol-AS-BI-phosphohydrolase, N-acetyl-β-glucosaminidase, and valine arylamidase is detected in strain ELA7T. In contrast, esterase (C4, is not able to produce indole and lacks the capability to reduce nitrate to nitrite. Menaquinone-7 (MK-7,, ELA7T (=KACC 23137T=JCM 36003T",Park et al. 2024,https://doi.org/10.1099/ijsem.0.006454,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006454%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691400034%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=RVclGZMiIi4JTZzzX2ZsPi8DE53fWVfwULQ0lLEvSK8%3D&reserved=0
10,[Methylobacterium amylolyticum ],"[OR473296, JAUYZI000000000]",is NEAU-140T (=MCCC 1K08801T=DSM 110568T,Zhang et al. 2024,https://doi.org/10.1099/

In [45]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
pub2_df

,PublishedName,Accessions,Strains,Authority,DOI,URL
0,[Neisseria leonii ],CP145606,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
1,[Neisseria leonii ],OQ121838,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,[Sphingobacterium tenebrionis ],PP092110,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
3,[Sphingobacterium tenebrionis ],JAYLLN000000000,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,[Phenylobacterium montanum ],PP386696,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
5,[Phenylobacterium montanum ],CP073078,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
6,[Phenylobacterium montanum ],CP073079,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com

In [46]:
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
pub4_df

,PublishedName,accession,Strains,Authority,DOI,URL
0,Neisseria leonii,CP145606,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
1,Neisseria leonii,OQ121838,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,Sphingobacterium tenebrionis,PP092110,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
3,Sphingobacterium tenebrionis,JAYLLN000000000,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,Phenylobacterium montanum,PP386696,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
5,Phenylobacterium montanum,CP073078,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
6,Phenylobacterium montanum,CP073079,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fww

In [47]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
df_unique = df_unique.dropna()
df_unique

,PublishedName,accession,Strains,Authority,DOI,URL
0,Neisseria leonii,CP145606,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
1,Neisseria leonii,OQ121838,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,Boutroux et al. 2024,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,Sphingobacterium tenebrionis,PP092110,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
3,Sphingobacterium tenebrionis,JAYLLN000000000,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",Zhang et al. 2024,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,Phenylobacterium montanum,PP386696,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
5,Phenylobacterium montanum,CP073078,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
6,Phenylobacterium montanum,CP073079,is S6T (=NBRC 115419T=GCMCC 1.18594T,Tang et al. 2024,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fww

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [48]:
df_unique['accession'] = df_unique['accession'].astype('str') 

In [49]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [50]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


0

In [51]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna()
srcchk_df 
#many need to add dropna here

,accession,NCBIname,taxid,strain
0,CP145606,Neisseria sp. 3986,2995413.0,3986
1,OQ121838,Neisseria sp. 3986,2995413.0,3986
2,PP092110,Sphingobacterium sp. PU5-4,3111775.0,PU5-4
3,JAYLLN000000000,Sphingobacterium sp. PU5-4,3111775.0,PU5-4
4,PP386696,Phenylobacterium montanum,2823693.0,S6
5,CP073078,Phenylobacterium montanum,2823693.0,S6
6,CP073079,Phenylobacterium montanum,2823693.0,S6
8,LC715209,Lacrimispora sp. YZC6,2962432.0,YZC6
9,BRYH00000000,Lacrimispora sp. YZC6,2962432.0,YZC6
10,OQ406256,Pedobacter sp.,1411316.0,ELA7


### Combine dataframes into one

In [54]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'Authority', 'taxid', 'DOI', 'URL' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,Authority,taxid,DOI,URL
0,Neisseria leonii,Neisseria sp. 3986,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,CP145606,3986,Boutroux et al. 2024,2995413.0,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
1,Neisseria leonii,Neisseria sp. 3986,of the species is 3986T (=R726T=CIP 109994T=LMG 32907T,OQ121838,3986,Boutroux et al. 2024,2995413.0,https://doi.org/10.1099/ijsem.0.006460,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006460%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691355255%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=Rde5ZrU00CRcas068Yqm1L5fnqXIlvON5h11PJ%2FvZ%2Fo%3D&reserved=0
2,Sphingobacterium tenebrionis,Sphingobacterium sp. PU5-4,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",PP092110,PU5-4,Zhang et al. 2024,3111775.0,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
3,Sphingobacterium tenebrionis,Sphingobacterium sp. PU5-4,", PU5-4T (=CGMCC 1.61908T=JCM 36663T",JAYLLN000000000,PU5-4,Zhang et al. 2024,3111775.0,https://doi.org/10.1099/ijsem.0.006455,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006455%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691367870%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=WshqYyTcQxzP0ZK5LItJAF%2BKfTo6v%2BJG318VvlI5Aow%3D&reserved=0
4,Phenylobacterium montanum,Phenylobacterium montanum,is S6T (=NBRC 115419T=GCMCC 1.18594T,PP386696,S6,Tang et al. 2024,2823693.0,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopiohZVa%2BNvbXDt7DscXB%2BVZshdEwsk9%2B2NkyrKE%3D&reserved=0
5,Phenylobacterium montanum,Phenylobacterium montanum,is S6T (=NBRC 115419T=GCMCC 1.18594T,CP073078,S6,Tang et al. 2024,2823693.0,https://doi.org/10.1099/ijsem.0.006463,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006463%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C79288cad63d84d4935b908dca7e26e48%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638569838691378606%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C40000%7C%7C%7C&sdata=nVQopioh

### write output to excel

In [55]:
combine_df.to_excel('NameCheckweek7.xlsx', engine='xlsxwriter', index = False, na_rep = '') 

### taxonerd test -- skip this

In [ ]:
from taxonerd import TaxoNERD
taxonerd = TaxoNERD(prefer_gpu=False)
nlp = taxonerd.load(model="en_ner_eco_md", exclude=[], linker="taxref", threshold=0.7)
nlp.pipe_names

### option to use biopython to retrieve orgnames based on accessions -- this approarch works but using srcchk for now so this isn't needed

In [ ]:
import Bio
from Bio import Entrez
from Bio import SeqIO

In [ ]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
x = SeqIO.read(handle, 'genbank')
x.annotations['organism']

### extra stuff not using, just saving for now

In [ ]:
combine_df['boo'] = combine_df['PublishedName'] != combine_df['NCBIname']

combine_df.style.applymap('background-color: red', subset=['PublishedName'])

combine_df

In [ ]:
 def color_positive_green(val):
    if combine_df.iloc['boo'] == True:
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color
 
combine_df.style.applymap(color_positive_green)

In [ ]:
#combine_df.apply_style_by_indexes(combine_df[combine_df['PublishedName'] != combine_df['NCBIname']], styler_obj=Styler(bg_color='red'), cols_to_style=['PublishedName', 'NCBIname'])

In [ ]:
#df.style.apply(color, axis=None).to_excel('styled.xlsx', engine='openpyxl')